In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import pandas as pd

In [ ]:
!which chromedriver

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/abby/.wdm/drivers/chromedriver/mac64/84.0.4147.30]


In [3]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [34]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []

# loop through all of the pages in this search
listOfPages = soup.find('ul', class_='paginator-list').find_all('li')
numberOfPages = listOfPages[0].find('a')['aria-label'].split()[3]
print(f"the number of pages to loop through is {numberOfPages}")

for i in range(1,int(numberOfPages)+1):
    
    # click on the next page number and scrape the html
    page_link = browser.links.find_by_text(i).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # loop through all of the products on that page
    products = soup.find_all('div', class_='search-result-gridview-item')
    print(f"quering page {i}...............")

    for product in products:

        # Get the link to the product page.
        link = product.find('a')
        href = link['href']
        product_url = base_url + href
        productURL_list.append(product_url)

        # Get the link to the product image.
        img = product.find('img')['src']
        productImage_list.append(img)

        # Get the product title.
        product_title = product.find('img')['alt']
        productTitle_list.append(product_title)

        # Try to get the number of reviews, there may not be any.
        try:
            review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
            reviewAmount_list.append(review_amount)

        except:
            review_amount = 0
            reviewAmount_list.append(review_amount)

        # In the case where there are at least one review...
        if int(review_amount) > 0:
            # Get the average number of stars
            stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
            starReview_list.append(stars_review)

            # Get the URL to the reviews section for that product.
            review_url = product.find('div', class_='stars').find('a')['href']
            reviewURL_list.append(base_url+review_url)

        # Otherwise, use defalt 0 or NaN values for these entries.
        else:
            stars_review = 0
            starReview_list.append(stars_review)

            review_url = "NaN"
            reviewURL_list.append(review_url)

        # Get the price of the product.
#         price = product.find('span', class_='price-main')
        price = product.find('span', class_='search-result-productprice')
        print(price)
        current_price = price.find('span', class_='visuallyhidden').text
        currentPrice_list.append(current_price)

    # Close the browser window
    # browser.quit()

    # Create a dictionary with the lists of the scrapped data.
    data = {
        "Title": productTitle_list,
        "URL": productURL_list,
        "Image": productImage_list,
        "AverageStars": starReview_list,
        "NumberofReviews": reviewAmount_list,
        "ReviewsURL": reviewURL_list,
        "Price": currentPrice_list
           }

    # Create a Pandas DataFrame with that dictionary
    product_df = pd.DataFrame.from_dict(data)
    product_df.head()

the number of pages to loop through is 14
quering page 1...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$14.98</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">14</span><span class="price-mark">.</span><span class="price-mantissa">98</span></span></span><span class="price-ppu-container"><span class="price-ppu-text" data-automation-id="price-ppu-text"></span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-1" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-pri

quering page 2...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$14.98</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">14</span><span class="price-mark">.</span><span class="price-mantissa">98</span></span></span><span class="price-ppu-container"><span class="price-ppu-text" data-automation-id="price-ppu-text"></span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-1" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-

quering page 3...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$539.00</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">539</span><span class="price-mark">.</span><span class="price-mantissa">00</span></span></span><span class="price-ppu-container"><span class="price-ppu-text" data-automation-id="price-ppu-text"></span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-1" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inlin

quering page 4...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$539.00</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">539</span><span class="price-mark">.</span><span class="price-mantissa">00</span></span></span><span class="price-ppu-container"><span class="price-ppu-text" data-automation-id="price-ppu-text"></span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-1" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inlin

<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-24" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$42.44</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">42</span><span class="price-mark">.</span><span class="price-mantissa">44</span></span></span><span class="price-ppu-container"><span class="price-ppu-text" data-automation-id="price-ppu-text"></span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-25" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="product-price-ppu"><span class="price display-inline-block arrange-fit price pric

...

quering page 11...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$148.00</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">148</span><span class="price-mark">.</span><span class="price-mantissa">00</span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-1" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$39.56</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-ch

quering page 12...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$18.55</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">18</span><span class="price-mark">.</span><span class="price-mantissa">55</span></span></span></span><span><span class="product-list-price-block"><span aria-hidden="true" class="secondary-price-prefix">List </span><span class="visuallyhidden">List Price</span><span class="price display-inline-block arrange-fit price price-secondary"><span class="visuallyhidden">$27.99</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">27</span><span class="price-mark">.</sp

<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-14" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$55.98</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">55</span><span class="price-mark">.</span><span class="price-mantissa">98</span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-15" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$56.16</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">56</span><span cl

quering page 13...............
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-0" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$93.97</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">93</span><span class="price-mark">.</span><span class="price-mantissa">97</span></span></span></span><span><span class="product-list-price-block"><span aria-hidden="true" class="secondary-price-prefix">List </span><span class="visuallyhidden">List Price</span><span class="price display-inline-block arrange-fit price price-secondary"><span class="visuallyhidden">$225.99</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">225</span><span class="price-mark">.</

<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-30" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$1294.00</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">1,294</span><span class="price-mark">.</span><span class="price-mantissa">00</span></span></span></span></span>
<span class="search-result-productprice gridview enable-2price-2" data-automation-id="item-tile-price-31" data-type="priceTags"><span class="visuallyhidden">Current Price</span><span class="price-main-block"><span class="price display-inline-block arrange-fit price price-main"><span class="visuallyhidden">$135.00</span><span aria-hidden="true" class="price-group"><span class="price-currency">$</span><span class="price-characteristic">135</span><

In [35]:
product_df = product_df.astype({
    "NumberofReviews": 'int',
    "AverageStars": 'float'
})

product_df.dtypes

Title               object
URL                 object
Image               object
AverageStars       float64
NumberofReviews      int64
ReviewsURL          object
Price               object
dtype: object

In [38]:
# product_df.describe()
product_df.head(6)

,Title,URL,Image,AverageStars,NumberofReviews,ReviewsURL,Price
0,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,https://i5.walmartimages.com/asr/ca7e3b65-d3c8...,4.3,541,https://www.walmart.com/ip/Holmes-Personal-Spa...,Current Price
1,Holmes Desktop HEPA Air Purifier with Visipure...,https://www.walmart.com/ip/Holmes-Desktop-HEPA...,https://i5.walmartimages.com/asr/22b30c76-a2a4...,4.4,81,https://www.walmart.com/ip/Holmes-Desktop-HEPA...,Current Price
2,Holmes TRUE HEPA Console Air Purifier with Fil...,https://www.walmart.com/ip/Holmes-TRUE-HEPA-Co...,https://i5.walmartimages.com/asr/0d248d12-4713...,4.4,208,https://www.walmart.com/ip/Holmes-TRUE-HEPA-Co...,Current Price
3,"Honeywell HEPA Allergen Remover Air Purifier, ...",https://www.walmart.com/ip/Honeywell-HEPA-Alle...,https://i5.walmartimages.com/asr/3574cc6b-93c9...,4.6,261,https://www.walmart.com/ip/Honeywell-HEPA-Alle...,Current Price
4,AUGIENB 60m² Large Room Air Purifier with 5 La...,https://www.walmart.com/ip/AUGIENB-60m-Large-R...,https://i5.walmartimages.com/asr/5f85d470-45c3...,5.0,1,https://www.walmart.com/ip/AUGIENB-60m-Large-R...,Current Price
5,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,https://i5.walmartimages.com/asr/5ba9740e-a3f1...,4.5,141,https://www.walmart.com/ip/High-Efficiency-HEP...,Current Price


In [45]:
print(product_df['ReviewsURL'][4])
print('----------------------------------------------------------------------------------------------------')
print(product_df['ReviewsURL'][5])

https://www.walmart.com/ip/AUGIENB-60m-Large-Room-Air-Purifier-5-Layer-True-HEPA-Filter-Negative-Ion-Cleaner-PM-2-5-Dust-Smoke-Allergies-Pets-Odor-Remover-Reduces-99-97-Viruses/655380111#customer-reviews
----------------------------------------------------------------------------------------------------
https://www.walmart.com/ip/High-Efficiency-HEPA-Type-Desktop-Air-Purifier-Carbon-Filter-110-Square-Foot-Room-Capacity-Three-Speed-HAP242-UC/14590699#customer-reviews


In [59]:
# LEFT OFF HERE
# WANT TO CLICK BY TEXT LIKE IN ABOVE QUERY, DIDN'T WORK THOUGH...
# THIS WILL LET ME GO TO EACH PAGE OF THE REVIEWS, 
# AND I THINK IT WILL LET ME LOAD ALL OF THAT PAGES REVIEWS

# Get reviews 

review_df = pd.DataFrame()

# loop through all of the products in the product dataframe
# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,6):
    if product_df['NumberofReviews'][i] > 0:
        
        productTitleList = []
        reviewURLList = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
    
#         try:
        browser.visit(product_df["ReviewsURL"][i])
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
        browser.visit(base_url+allReviewsURL)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        
        # loop through all of the pages in this search
        reviewPages = soup.find('ul', class_='paginator-list').find_all('li')
        pagesOfReviews = reviewPages[-1].find('button').text
        print(f"the number of pages to loop through is {pagesOfReviews}")
        for j in range(1, int(pagesOfReviews)+1):
            
            page_link = browser.links.find_by_text(j).click()
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            reviews = soup.find_all('div', class_='ReviewList-content')

            for review in reviews:

                productTitleList.append(product_df['Title'][i])
                reviewURLList.append(product_df['ReviewsURL'][i])

                try:
                    title = review.find('h3', class_='review-title').text
                    reviewTitleList.append(title)
                    print(title)

                except:
                    title = 'None'
                    reviewTitleList.append(title)
                    print(title)
#                         pass

#                 # adding try only because the browser needs to scroll
#                 # once this is added, there wont need to be a try
#                 # each review has to have a star value
#                 try:
#                     reviewRating = review.find('span', class_='seo-avg-rating').text
#                     reviewRatingList.append(reviewRating)
#                     print(reviewRating)
#                 except:
#                     pass

#                 # adding try and except for the review body for same reason as stars
#                 try:
#                     reviewComment = review.find('p').text
#                     reviewCommentList.append(reviewComment)
# #                         print(reviewComment)
#                 except:
#                     pass


#             # only until I fix the scroll
#             reviewTitleList.pop()
#             productTitleList.pop()
#             reviewURLList.pop()

#             data = {
#                 'ProductTitle': productTitleList,
#                 'ReviewURL': reviewURLList,
#                 'ReviewTitle': reviewTitleList,
#                 'ReviewStarRating': reviewRatingList,
#                 'ReviewComment': reviewCommentList
#             }
#             print('stored data')
#             productReviews_df = pd.DataFrame.from_dict(data)
#             productReviews_df.head()
#             print('into dataframe')

#         except:
#             pass
    
#         print('add to existing dataframe')
#         review_df = review_df.append(productReviews_df, ignore_index=True)
# print('done')
# #                 browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

the number of pages to loop through is 1


ElementDoesNotExist: no elements could be found with link by text "1"

In [40]:
# len(review_df)
# review_df.tail()
review_df.describe()

,ProductTitle,ReviewURL,ReviewTitle,ReviewStarRating,ReviewComment
count,11,11,11,11,11
unique,2,2,11,3,11
top,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,Great air purifier,5.0,Air purifier works great. Comes with one filte...
freq,10,10,1,9,1


In [ ]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('span', class_='seo-avg-rating').text
# starts = reviews.find('span', class_='seo-average-rating')
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

In [ ]:
reviews = soup.find_all('div', class_='ReviewList-content')

In [ ]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,5):
    if product_df['NumberofReviews'][i] > 0:
#     number_of_reviews = product_df['NumberofReviews'][i]
#     for j in range(number_of_reviews):
        
        productTitleList = []
        reviewURLList = []
        reviewSiteLink = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
#         if len(reviewRatingList) < product_df['NumberofReviews'][i]:
        
#         while len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
        try:
            # go to the product url link
            browser.visit(product_df["ReviewsURL"][i])
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the 'see all reviews' button and go there
            allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
            browser.visit(base_url+allReviewsURL)
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the list of reviews
            reviews = soup.find_all('div', class_='ReviewList-content')
            
#             for review in reviews:
            for j in range(len(reviews)):
                reviewSiteLink.append(review)
                reviewSiteLink[j].send_keys(Keys.PAGE_DOWN)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find('div', class_='ReviewList-content')
                
#                 print(f"j={j}")
                productTitleList.append(product_df['Title'][i])
                reviewURLList.append(product_df['ReviewsURL'][i])
                title, stars, comments = scrapeReviews(review)
                reviewTitleList.append(title)
                reviewRatingList.append(stars)
                reviewCommentList.append(comments)


                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                productReviews_df = pd.DataFrame.from_dict(data)
                productReviews_df.head()
                print('into dataframe')

        except:
            pass

        print('add to existing dataframe')
        review_df = review_df.append(productReviews_df, ignore_index=True)
print('done')

In [ ]:
# len(review_df)
# review_df.tail()
review_df.describe()

In [ ]:
def scrapeReviews(review):
    
    # get the title of the review
    try:
        title = review.find('h3', class_='review-title').text
    except:
        title = 'None'
        
    # get the number of stars in this review
    reviewRating = review.find('span', class_='seo-avg-rating').text
    
    # get the context of the review
    reviewComment = review.find('p').text
        
    # return these items as a tuple
    return title, stars, comments

In [ ]:
browser.quit()

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")